In [1]:
!hostname

midway3-0277.rcc.local


# Imports/Setpu

In [ ]:
# import sys
# import os
# import shutil

# username = os.environ.get('USER')
# scratch_base = f"/scratch/midway3/{username}/fea_project"
# custom_lib_path = os.path.join(scratch_base, "my_custom_libs")
# py_version = f"python{sys.version_info.major}.{sys.version_info.minor}"
# site_packages = os.path.join(custom_lib_path, "lib", py_version, "site-packages")

# # Clean up old installation
# if os.path.exists(custom_lib_path):
#     print(f"Removing old libraries...")
#     shutil.rmtree(custom_lib_path)

# install_cmd = (
#     f"pip install --target='{site_packages}' "
#     "--upgrade "
#     "--no-cache-dir "
#     "'numpy<2.0' "
#     "'pandas' "
#     "'bottleneck' "
#     "'numexpr' "
#     "'torch' "
#     "'torchvision' "
#     "'accelerate>=0.31.0' "
#     "'tensorflow' "
#     "'tf-keras' "
#     "'transformers' "
#     "'sentence-transformers' "
#     "'datasets' "
#     "'botocore' "
#     "'aiobotocore' "
#     "'s3fs' "
#     "'openpyxl' "
#     "'sentencepiece' "
#     "'plotly' "
#     "'scikit-learn' "
#     "'papermill' "
#     "'scrapbook' "
#     "'openai' "
#     "'pydantic' "
#     "'tqdm' "
# )

# print("Installing with NumPy <2.0...")
# exit_code = os.system(install_cmd)

# if exit_code == 0:
#     print("\nSUCCESS: Complete stack installed.")
# else:
#     print(f"\nERROR: Installation failed with code {exit_code}")

# print("\n##### RESTART KERNEL NOW #####")

In [2]:
import sys
import os

### Installs all the packages in the install library (my_custom_libs) ##

# --- 1. SETUP PATHS FIRST (you are going to need to change to whatever your path to the instal library is in) ---
username = os.environ.get('USER')
scratch_base = f"/scratch/midway3/{username}/fea_project"
custom_lib_path = os.path.join(scratch_base, "my_custom_libs")
hf_cache_path = os.path.join(scratch_base, "hf_cache")

py_version = f"python{sys.version_info.major}.{sys.version_info.minor}"
site_packages = os.path.join(custom_lib_path, "lib", py_version, "site-packages")
sys.path.insert(0, site_packages)

# FORCE custom path to the FRONT of the list
if site_packages not in sys.path:
    sys.path.insert(0, site_packages)

print(f"Python is looking here first: {sys.path[0]}")

# --- 2. ENV VARIABLES ---
os.environ['HF_HOME'] = hf_cache_path
os.environ['TRANSFORMERS_CACHE'] = hf_cache_path
os.environ['PYTHONUSERBASE'] = custom_lib_path

# --- 3. NOW IMPORT LIBRARIES ---
# Only import AFTER sys.path is updated
import datasets 
import sentence_transformers
from sentence_transformers import SentenceTransformer

# --- 4. VERIFY PATHS ---
print(f"Datasets is loaded from: {os.path.dirname(datasets.__file__)}")

if "software" in os.path.dirname(datasets.__file__):
    print("FAILURE: Still loading system datasets.")
else:
    print("SUCCESS: All libraries isolated in scratch.")

Python is looking here first: /scratch/midway3/None/fea_project\my_custom_libs\lib\python3.13\site-packages


KeyboardInterrupt: 

In [79]:
import sys
import os
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)
    print(f"Added current directory to sys.path: {current_dir}")
import papermill as pm
import scrapbook as sb
import pandas as pd
import os
from datetime import datetime
import importlib
import free_entailments_algorithm_utils as fea

# Global Variables:

In [80]:
#Change to False if running the real thing.

test = False

In [81]:

importlib.reload(fea)

#List of all sentences/sentence ids being used
df_p = pd.read_excel("ClauseLevel_df_p.xlsx")
df_sc = pd.read_excel("ClauseLevel_df_sc.xlsx")

In [82]:
## All clauses. Of the form: argument_id	sentence_id	sentence

df_clause = pd.concat([df_p,df_sc]).drop_duplicates(subset='sentence_id')

# Generate initial unlabeled pairs (production mode only).
# We generate up to 50M pairs (BB + BS), excluding any already-labeled pairs.
# These are randomly sampled from the ~450M valid candidate space.
# seen_indices_dir persists consumed pair indices across rounds so
# subsequent rounds never re-sample pairs from earlier rounds.

if not test:
    df_pairs = fea.generate_valid_pairs(
        df_p,
        df_sc,
        'sentence_id',
        'sentence',
        max_pairs=1_000_000,
        exclude_labeled_csv="labeled_pairs/llm_labeled_pairs.csv",
        seen_indices_dir="seen_pair_indices",
    )
    print(f"Generated {len(df_pairs):,} unlabeled pairs (after excluding already-labeled)")


=== Generating premise-premise pairs ===
  Dataset size: 49155 (9564 books, 39591 speeches)
  Max possible valid pairs: 424,378,590
  Seen-index tracking: 0 BB + 0 BS already consumed (45,730,266 BB + 378,648,324 BS remaining)
  Sampling 762,755 pairs from 424,378,590 possible...
    BB: sampled 82,193 pairs (seen 0, avail 45,730,266)
    BS: sampled 680,562 pairs (seen 0, avail 378,648,324)
  Generated 762,755 unique pairs

=== Generating conclusion-conclusion pairs ===
  Dataset size: 15289 (2511 books, 12778 speeches)
  Max possible valid pairs: 35,236,863
  Seen-index tracking: 0 BB + 0 BS already consumed (3,151,305 BB + 32,085,558 BS remaining)
  Sampling 237,245 pairs from 35,236,863 possible...
    BB: sampled 21,217 pairs (seen 0, avail 3,151,305)
    BS: sampled 216,028 pairs (seen 0, avail 32,085,558)
  Generated 237,245 unique pairs

=== SUMMARY ===
Premise-premise pairs: 762,755
Conclusion-conclusion pairs: 237,245
Total valid pairs: 1,000,000
Columns: ['id1', 'id2']
Filt

In [66]:
# # Creates Embedding Cache (If not already created) from df_clause:
# import pickle

# cache_path = "embedding_cache_finetuned.pkl"

# if os.path.exists(cache_path):
#     with open(cache_path, 'rb') as f:
#         embedding_cache_finetuned = pickle.load(f)
#     print(f"Loaded existing embedding cache: {len(embedding_cache_finetuned)} embeddings")
# else:
#     # Create cache from all unique clauses
#     embedding_cache_finetuned = fea.create_embedding_cache(
#         df_texts=df_clause,
#         id_col='sentence_id',
#         text_col='sentence',
#         model_name="BAAI/bge-large-en-v1.5",
#         batch_size=128,
#         show_progress_bar=True
#     )
#     # Save to disk for future runs
#     with open(cache_path, 'wb') as f:
#         pickle.dump(embedding_cache_finetuned, f)
#     print(f"Saved embedding cache to {cache_path}")


In [83]:
input_file = "labeled_pairs/Results_DS_BtoS_iteration_0.csv"
df_input = pd.read_csv(input_file)

if test:
    importlib.reload(fea)
    df_input = pd.read_csv(input_file)
    
    input_file, remaining_llm_calls = fea.two_random_subsamples(
        df = df_input,
        frac1 = 0.8,
        frac2 = 0.2,
        random_state = 42
    )
else:
    remaining_llm_calls = None
    # Use Results_DS_BtoS_iteration_0 as labeled input,
    # and subtract already-labeled pairs from the 20M unlabeled pool
    df_entailed_pairs = df_input.rename(columns={'sentence_id_1': 'id1', 'sentence_id_2': 'id2'})
    unlabeled_pairs = fea.setminus(df_pairs, df_entailed_pairs, ["id1", "id2"])
    print(f"Unlabeled pairs after removing labeled: {len(unlabeled_pairs):,}")


Set difference: 1,000,000 - 10 = 1,000,000 rows
Unlabeled pairs after removing labeled: 1,000,000


In [84]:
import pickle
with open("embedding_cache_finetuned.pkl", 'rb') as f:
    embedding_cache_finetuned = pickle.load(f)
print(f"Loaded embedding cache: {len(embedding_cache_finetuned)} embeddings")

Loaded embedding cache: 63909 embeddings


In [ ]:
num_iterations = 3          # hard cap (set high – budget is the real limiter)
iteration_stats = []

sent_frac = 0.5
budget = 0.0

# ---- Budget controls ----
# The loop stops automatically once cumulative LLM spend reaches budget_dollars.
# Set to None for unlimited (loop runs until num_iterations).
budget_dollars = 0.10       
cost_per_1k_pairs = 2.0       # ~$2 per 1,000 pairs (DeepSeek R1)

# DeepSeek API key for production LLM calls
api_key = os.environ.get("DEEPSEEK_API_KEY", "sk-")

# FEA Loop using Papermill

This notebook uses papermill to iteratively run the FEA_Pipeline notebook, feeding the output of each iteration as input to the next.

In [87]:
output_dir = "fea_iterations"
os.makedirs(output_dir, exist_ok=True)

start_iteration = 0

In [88]:
print(f"Starting FEA Loop with {num_iterations} iterations")
print("="*60)


Starting FEA Loop with 3 iterations


In [89]:
importlib.reload(fea)

iteration_stats = fea.run_fea_loop(
    test=test,
    input_file=input_file,
    df_clause=df_clause,
    embedding_cache=embedding_cache_finetuned,
    num_iterations=num_iterations,
    start_iteration=start_iteration,
    output_dir=output_dir,
    sent_frac=sent_frac,
    budget=budget,
    remaining_llm_calls=remaining_llm_calls,
    unlabeled_pairs=unlabeled_pairs if 'unlabeled_pairs' in dir() else None,
    deepseek_api_key=api_key,
    budget_dollars=budget_dollars,
    cost_per_1k_pairs=cost_per_1k_pairs,
)

VALIDATING INITIAL DATA STATE
Initial labeled pairs: 10
Initial unlabeled pairs: 1000000
Total: 1000010
✓ Cleaned up old loop data directory
Starting with 10 already labeled pairs
Unlabeled pool: 1000000
Total pairs: 1000010
Budget: $0.10  ($2.00 / 1k pairs)

ITERATION 0/3
Status: Labeled=10, Unlabeled=1000000, Total=1000010
Cost so far: $0.00  ($0.10 remaining)
Executing FEA_Pipeline.ipynb with input: labeled_pairs/Results_DS_BtoS_iteration_0.csv


Executing:   0%|          | 0/26 [00:00<?, ?cell/s]


✓ Retrieved df_to_llm from scrapbook
  Shape: (50, 7)
  Columns: ['sentence_id_2', 'sentence_id_1', 'sentence_text_2', 'argument_id_2', 'sentence_text_1', 'argument_id_1', 'score']
✓ Saved df_to_llm to fea_iterations\loop_data/df_to_llm_iter_0.csv

EXECUTING LLM EVALUATOR (forward direction)
Sending 50 pairs to LLM...


Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

✓ Forward LLM evaluation complete
✓ LLM labeled pairs updated: 62 total in labeled_pairs/llm_labeled_pairs.csv

ONE-WAY RESULTS PROCESSING
Total input pairs: 50
Resolved rows: 56 (YES=0, NO=56)
  ↳ Inferred reverse NO (money saved): 28
Need reverse LLM call: 22

✓ LLM labeled pairs updated: 90 total in labeled_pairs/llm_labeled_pairs.csv
✓ Recorded 28 inferred reverse-NO pairs in labeled_pairs/llm_labeled_pairs.csv

SENDING 22 REVERSE PAIRS TO LLM
Model: deepseek-reasoner
Input: labeled_pairs\reverse_iter_0_input.csv
Output: labeled_pairs\reverse_iter_0_output.csv
Loading data from labeled_pairs\reverse_iter_0_input.csv...
Loading data from ArgLevel_ClauseIds_df.xlsx...
Loaded 22 sentence pairs
Using model: deepseek-reasoner
Using prompt type: test_prompt_tot_json2
Running batch evaluation...


100%|██████████| 22/22 [00:00<00:00, 47.32it/s]


[DEBUG] content length: 1505, reasoning_content length: 5191
[DEBUG] content preview: {
  "sentence_id_1": "B0258001sc",
  "sentence_id_2": "B0126001sc",
  "answers": "YES, YES, YES",
  "reasoning": "1. Logically, if sovereign power requires the consent of the people (Statement 1), then any legitimate holder of that power—i.e., rulers—must derive their legitimacy from that same conse
[DEBUG] reasoning preview: We are given two statements derived from two arguments. Statement 1: "Sovereign power requires the consent of the people." Statement 2: "Rulers derive their legitimacy from the consent of the governed." We need to assess whether Statement 1 entails Statement 2. Entailment means that if Statement 1 i
Saving progress at batch 1...
Saving progress at batch 2...
Deleting previous file: labeled_pairs\reverse_iter_0_output_progress_batch_1.csv
Saving progress at batch 3...
Deleting previous file: labeled_pairs\reverse_iter_0_output_progress_batch_2.csv
Saving progress at batch 4...
Del

In [62]:
stats_df = fea.save_iteration_stats(iteration_stats, output_dir)


Iteration Statistics:
   iteration  labeled_input  unlabeled_available  pairs_selected  \
0          0              6                 1000               1   
1          1              7                  999               1   

   cumulative_labeled  unlabeled_remaining  \
0                   7                  999   
1                   8                  998   

                                     output_file  entailment_ratio  \
0  labeled_pairs/Results_DS_BtoS_iteration_1.csv               0.0   
1  labeled_pairs/Results_DS_BtoS_iteration_2.csv               0.0   

   iteration_cost  cumulative_cost  
0           0.002            0.002  
1           0.002            0.004  

COST SUMMARY
Total pairs sent to LLM: 2
Total cost: $0.00

✓ Saved iteration statistics to fea_iterations\iteration_stats.csv
